Generate images

# STARBURST pattern caused by resname = SSN

In [155]:
import numpy as np
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
import image_box as image
import importlib

import os
import operator
import xlrd

In [156]:
importlib.reload(image)

<module 'image_box' from 'C:\\Users\\Thomas\\wlcsim_membrane\\visualization\\image_box.py'>

In [157]:
width = 60
gen = 4
l_box = 20 
if width == 60:
    ind0 = 37631
    indf = 37752
elif width == 75:
    ind0 = 37616
    indf = 37767
elif width == 90:
    ind0 = 37601
    indf = 37782

In [158]:
base = "C:\\users/Thomas/wlcsim_membrane/visualization/image_data/"
r_files_path = base + "r_files/"
rephist_path = base + "finalrep.xlsx"

In [159]:
# load in rep history chis
wb = xlrd.open_workbook(rephist_path)
ws = wb.sheet_by_index(0)
chi_val_list = ws.col_values(4)[1:]

In [160]:
#chi_val_list

In [161]:
'''PREPROCESSING:
r files- copy all into one folder
get chi values from same repHistory (copy last exchange into new notepad, import into excel, save)
'''
  
# Change the directory
os.chdir(r_files_path)

r_df_list = []
v_val_list = []
for file in os.listdir():
    file_path = f"{r_files_path}\{file}"
    
    if file_path[-3] != "pdb":
        v_val = file_path[-2:]
        if v_val[0] == "v":
            v_val = v_val[1]
        v_val = float(v_val) # smallest v = smallest chi

        file = open(file_path)
        r_poly_raw = file.readlines()
        file.close()
        n_pt = len(r_poly_raw)
        r_poly = np.zeros((n_pt,3))
        meth_seq = np.zeros(n_pt)
        hp1_seq = np.zeros(n_pt)
        for i in range(n_pt):
            line = r_poly_raw[i].split()
            r_poly[i, 0] = float(line[0])# - np.floor(float(line[0])/l_box) * l_box
            r_poly[i, 1] = float(line[1])#- np.floor(float(line[1])/l_box) * l_box
            r_poly[i, 2] = float(line[2])#- np.floor(float(line[2])/l_box) * l_box
            #hp1_seq[i] = float(line[3])
            meth_seq[i] = float(line[3])  

        #create box adjust array to prevent connections across periodic boundary
        box_adjust_left = np.ones(n_pt)
        box_adjust_right = np.ones(n_pt)

        x_adjust = np.floor(r_poly[:,0]/l_box)
        y_adjust = np.floor(r_poly[:,1]/l_box)
        z_adjust = np.floor(r_poly[:,2]/l_box)

        for i in range(n_pt):
            if i == 0:
                if (x_adjust[i+1] != x_adjust[i]) or (y_adjust[i+1] != y_adjust[i]) or (z_adjust[i+1] != z_adjust[i]):
                    box_adjust_right[i] = 0
            elif i == (n_pt - 1):
                if (x_adjust[i-1] != x_adjust[i]) or (y_adjust[i-1] != y_adjust[i]) or (z_adjust[i-1] != z_adjust[i]):
                    box_adjust_left[i] = 0
            else:
                if (x_adjust[i+1] != x_adjust[i]) or (y_adjust[i+1] != y_adjust[i]) or (z_adjust[i+1] != z_adjust[i]):
                    box_adjust_right[i] = 0
                if (x_adjust[i-1] != x_adjust[i]) or (y_adjust[i-1] != y_adjust[i]) or (z_adjust[i-1] != z_adjust[i]):
                    box_adjust_left[i] = 0
        r_poly_box = np.zeros((n_pt,3))    
        r_poly_box[:,0] = r_poly[:,0] - np.floor(r_poly[:,0]/l_box) * l_box
        r_poly_box[:,1] = r_poly[:,1] - np.floor(r_poly[:,1]/l_box) * l_box
        r_poly_box[:,2] = r_poly[:,2] - np.floor(r_poly[:,2]/l_box) * l_box

        r_poly_art = r_poly_box[ind0:indf, :]
        hp1_seq_art = hp1_seq[ind0:indf]
        meth_seq_art = meth_seq[ind0:indf]

        file = 'r_poly_w' + str(width) + 'g' + str(gen) + "_chi=" + str(chi_val_list[int(v_val-1)]) + '.pdb'
        #works
        image.gen_pymol_file(r_poly, meth_seq, limit_n = True, n_max = int(n_pt / 2), add_com = False, filename = file,
                             box_adjust_l = box_adjust_left, box_adjust_r = box_adjust_right)


    
    
#     name_list = ["X", "Y", "Z", "Type"]        
#     file_df = pd.read_csv(file_path, names = name_list, delim_whitespace=True)#, delimiter = " ")#
    
#     r_df_list_orig.append([v_val, file_df])
#     v_val_list_orig.append(v_val)
#     del file_df
# r_df_list_orig = sorted(r_df_list_orig, key=operator.itemgetter(0))
# v_val_list_orig = sorted(v_val_list_orig)

# wb = xlrd.open_workbook(rephist_path_orig)
# ws = wb.sheet_by_index(0)
# chi_val_list_orig = ws.col_values(4)[1:]

# END OF FILE

In [105]:
#file = open("data/configurations/art_demeth_seghalf_" + str(width) + "/gen" + str(gen) + "/r85v0", "r")

# #edited, chi = 1
# file = open("C:\\users/Thomas/wlcsim_membrane/data/var_solv/r17v62_2000")



# #edited, chi = 0
# file = open("C:\\users/Thomas/wlcsim_membrane/data/AB_solution/edited/5.15.2021_ABsolution_EDITED_rigid_kappa=20_1000poly_24hrs/all_rs/r14v1")


# #unedited, chi = 1
# file = open("C:\\users/Thomas/wlcsim_membrane/data/AB_solution/unedited/5.12.2021_ABsolution_ORIGINAL_rigid_kappa=20_1000poly_24hrs/all_rs/r15v62")

#unedited, chi = 0
#file = open("C:\\users/Thomas/wlcsim_membrane/data/AB_solution/unedited/5.12.2021_ABsolution_ORIGINAL_rigid_kappa=20_1000poly_24hrs/all_rs/r15v1")


In [106]:
#edited, chi = 0
# file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution/1000_polymers/edited/X=0.00_lam=0_rigid/all_rs/r22v1")

# #edited, chi = 1
# file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution/1000_polymers/edited/X=1.00_lam=0_rigid/all_rs/r22v13")

# #edited, chi = 2
# file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution/1000_polymers/edited/X=2.00_lam=0_rigid/all_rs/r22v25")

# #edited, chi = 3
# file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution/1000_polymers/edited/X=3.00_lam=0_rigid/all_rs/r22v37")

# #edited, chi = 4
# file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution/1000_polymers/edited/X=4.00_lam=0_rigid/all_rs/r22v49")

# #edited, chi = 5
# file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution/1000_polymers/edited/X=5.00_lam=0_rigid/all_rs/r22v61")


In [107]:
# ab melt flexible ~alternating polymer 

#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_melt/X=4.00_lam=-.5_flexible/r27v74")

In [108]:
# Hanley 2000

# x_bs = 0.4
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000/X_bs=0.40_flex/all_rs/r18v25")

# X_bs = 0.5
file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000/X_bs=0.50_flex/all_rs/r20v31")

# x_bs = 0.6
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000/X_bs=0.60_flex/all_rs/r18v37")

# x_bs = 0.8
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000/X_bs=0.80_flex/all_rs/r18v49")

# x_bs = 1.0
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000/X_bs=1.0_flex/all_rs/r18v61")


In [109]:
# Hanley 2000 exflex Nm=50

# x_bs = 0.4
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000_Nm=50/X_bs=0.40_flex/all_rs/r20v25")

# X_bs = 0.5
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000_Nm=50/X_bs=0.50_flex/all_rs/r20v25")

# x_bs = 0.6
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000_Nm=50/X_bs=0.60_flex/all_rs/r20v37")

# x_bs = 0.8
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000_Nm=50/X_bs=0.80_flex/all_rs/r20v49")

# x_bs = 1.0
#file = open("C:\\users/Thomas/wlcsim_membrane/data/struc_fac/AB_solution_3_chis/1000_polymers/new_selective/Hanley_2000_Nm=50/X_bs=1.0_flex/all_rs/r20v61")


In [110]:
width = 60
gen = 4
l_box = 20 
if width == 60:
    ind0 = 37631
    indf = 37752
elif width == 75:
    ind0 = 37616
    indf = 37767
elif width == 90:
    ind0 = 37601
    indf = 37782

r_poly_raw = file.readlines()
n_pt = len(r_poly_raw)
r_poly = np.zeros((n_pt,3))
meth_seq = np.zeros(n_pt)
hp1_seq = np.zeros(n_pt)
for i in range(n_pt):
    line = r_poly_raw[i].split()
    r_poly[i, 0] = float(line[0])# - np.floor(float(line[0])/l_box) * l_box
    r_poly[i, 1] = float(line[1])#- np.floor(float(line[1])/l_box) * l_box
    r_poly[i, 2] = float(line[2])#- np.floor(float(line[2])/l_box) * l_box
    #hp1_seq[i] = float(line[3])
    meth_seq[i] = float(line[3])  


In [111]:
#create box adjust array to prevent connections across periodic boundary
box_adjust_left = np.ones(n_pt)
box_adjust_right = np.ones(n_pt)

x_adjust = np.floor(r_poly[:,0]/l_box)
y_adjust = np.floor(r_poly[:,1]/l_box)
z_adjust = np.floor(r_poly[:,2]/l_box)

for i in range(n_pt):
    if i == 0:
        if (x_adjust[i+1] != x_adjust[i]) or (y_adjust[i+1] != y_adjust[i]) or (z_adjust[i+1] != z_adjust[i]):
            box_adjust_right[i] = 0
    elif i == (n_pt - 1):
        if (x_adjust[i-1] != x_adjust[i]) or (y_adjust[i-1] != y_adjust[i]) or (z_adjust[i-1] != z_adjust[i]):
            box_adjust_left[i] = 0
    else:
        if (x_adjust[i+1] != x_adjust[i]) or (y_adjust[i+1] != y_adjust[i]) or (z_adjust[i+1] != z_adjust[i]):
            box_adjust_right[i] = 0
        if (x_adjust[i-1] != x_adjust[i]) or (y_adjust[i-1] != y_adjust[i]) or (z_adjust[i-1] != z_adjust[i]):
            box_adjust_left[i] = 0

In [112]:
# example of box adjust
ls = [0, 0, 0, 1, 1, 0] # example x_adjust array
adjust_l = np.ones(6)
adjust_r = np.ones(6)
for i in range(len(ls)):
    if i == 0:
        if ls[i+1] != ls[i]:
            adjust_r[i] = 0
    elif i == (len(ls) - 1):
        if ls[i-1] != ls[i]:
            adjust_l[i] = 0
    else:
        if ls[i+1] != ls[i]:
            adjust_r[i] = 0
        if ls[i-1] != ls[i]:
            adjust_l[i] = 0

print(adjust_l)
print(adjust_r)

[1. 1. 1. 0. 1. 0.]
[1. 1. 0. 1. 0. 1.]


In [113]:
r_poly_box = np.zeros((n_pt,3))    
r_poly_box[:,0] = r_poly[:,0] - np.floor(r_poly[:,0]/l_box) * l_box
r_poly_box[:,1] = r_poly[:,1] - np.floor(r_poly[:,1]/l_box) * l_box
r_poly_box[:,2] = r_poly[:,2] - np.floor(r_poly[:,2]/l_box) * l_box

r_poly_art = r_poly_box[ind0:indf, :]
hp1_seq_art = hp1_seq[ind0:indf]
meth_seq_art = meth_seq[ind0:indf]

In [114]:
r_poly = r_poly_box

Rotate the frame to the com of the artificial region- SKIPPED

In [115]:
# r_com = np.mean(r_poly, axis= 0)
# r_poly = r_poly - r_com
# r_poly_art = r_poly_art - r_com

# r_com_art = np.mean(r_poly_art, axis= 0)
# t_3 = r_com_art / np.sqrt(np.sum(r_com_art ** 2))
# t_1 = np.array([1, 0, 0])
# t_1 = t_1 - np.dot(t_1, t_3) * t_3
# t_1 = t_1 / np.sqrt(np.sum(t_1 ** 2))
# t_2 = np.cross(t_3, t_1)

# r_poly_rot = np.zeros((n_pt, 3))
# r_poly_art_rot = np.zeros((len(r_poly_art[:,0]), 3))

# for i_b in range(n_pt):
#     r_poly_rot[i_b, 0] = np.dot(r_poly[i_b, :], t_1)
#     r_poly_rot[i_b, 1] = np.dot(r_poly[i_b, :], t_2)
#     r_poly_rot[i_b, 2] = np.dot(r_poly[i_b, :], t_3)

# for i_b in range(len(r_poly_art[:,0])):
#     r_poly_art_rot[i_b, 0] = np.dot(r_poly_art[i_b, :], t_1)
#     r_poly_art_rot[i_b, 1] = np.dot(r_poly_art[i_b, :], t_2)
#     r_poly_art_rot[i_b, 2] = np.dot(r_poly_art[i_b, :], t_3)

# r_poly = r_poly_rot
# r_poly_art = r_poly_art_rot

In [116]:
len(r_poly[:,0])

40000

Create the conformation of the confining circle- SKIPPED

In [117]:
# n_circle = 200
# r_circle = np.zeros((n_circle,3))
# r_com = np.mean(r_poly, axis= 0)
# r_com[1] = r_com[1] + 0.85
# r_com[2] = r_com[2] - 0.75
# r_com_0 = r_com
# r_com[1] = (np.min(r_poly[:,1]) + np.max(r_poly[:,1])) / 2
# r_com[2] = (np.min(r_poly[:,2]) + np.max(r_poly[:,2])) / 2
# radius = 31 #np.max(np.sqrt(np.sum((r_poly - r_com) ** 2, axis = 1)))
# for i in range(n_circle):
#     r_circle[i, 0] = r_com[0]
#     r_circle[i, 1] = r_com[1] + radius * np.cos(2 * np.pi * i / n_circle)
#     r_circle[i, 2] = r_com[2] + radius * np.sin(2 * np.pi * i / n_circle)    

In [118]:
file = 'r_poly_w' + str(width) + 'g' + str(gen) + '.pdb'
#works
image.gen_pymol_file(r_poly, meth_seq, limit_n = True, n_max = int(n_pt / 2), add_com = False, filename = file,
                     box_adjust_l = box_adjust_left, box_adjust_r = box_adjust_right)

file = 'r_poly_art_w' + str(width) + 'g' + str(gen) + '.pdb'
#image.gen_pymol_file(r_poly_art, meth_seq_art, hp1_seq_art, limit_n = False, filename=file)
file = 'r_circle_w' + str(width) + 'g' + str(gen) + '.pdb'
#image.gen_pymol_file(r_circle, limit_n = False, filename=file, ring = True)

In [119]:
len(meth_seq[meth_seq == 1])

20000